In [1]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [6]:
from pathlib import Path
result = list(Path("./basefiles").rglob("*.[tjJ][pP][gG]"))
result

[]

In [29]:
import os
result = [os.path.join(dp, f) for dp, dn, filenames in os.walk("./basefiles") for f in filenames if os.path.splitext(f)[1].lower() in [".jpg",".gif",".png",".tga"]]
result

import numpy as np
arrs = np.array_split(result, 20)
arrs[0]



array(["./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_52_1.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_52_2.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_53_1.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_53_2.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_53_4.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_53_5.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_56_1.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_57_1.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_57_2.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_58_1.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_58_2.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_58_3.jpg",
       "./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H00

In [30]:
os.path.splitext("./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_52_1.jpg")

("./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_52_1", '.jpg')

In [32]:
os.path.dirname("./basefiles/DC72 H (Objectfoto's)/Aardewerk/DC72_P01_H0001_52_1.jpg")

"./basefiles/DC72 H (Objectfoto's)/Aardewerk"

In [38]:
for root, dirs, files in os.walk("./basefiles/fotos/", topdown=False):
    print(root)

./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Aardewerk
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Bot (Dierlijk)
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Bot (Menselijk)
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Bot (Vis)
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Glas
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Hout
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Leer
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Natuursteen
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Pijpaarden Beeldjes
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Pijpen
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Schelp
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Textiel
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)/Varia
./basefiles/fotos/DB34 H (Objectfoto's)/DB34 H (Objectfoto's)
./base